# Data Preprocessing

The following code reads in the data, and saves it in numpy arrays for later use

In [1]:
import numpy as np
import pandas as pd
from netCDF4 import Dataset
import datetime
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

Iterate through all of the files in the data folder, and extract the desired variable. In this case, that variable is the lattitude and logintude grid of that value. For example, the air temperature data is a grid of temperatures, corresponding to different locations on the globe. From each value, 16 years of data are read in and stored by year.

In [26]:
import os

folder = "C:/users/zdunkerton/documents/githubdata/hurricane/initial_data"
for dir in os.listdir(folder):
    data = []
    time = []
    for filename in os.listdir(folder+"/"+dir):
        x = Dataset(folder+"/"+dir+"/"+filename,'r').variables[filename.split('.')[0]]
        y = Dataset(folder+"/"+dir+"/"+filename,'r').variables["time"]
        if(x.shape[0]>1460):
            x =x[:1460,:,:]
            y = y[:1460]
        time.append(y)
        data.append(x)
    time = np.array(time)
    time = np.reshape(time, (23360,))
    data = np.array(data)
    time_name = "C:/Users/zdunkerton/Documents/GitHubData/Hurricane/Preprocessed/time.npy"
    data_name = "./"+dir+".npy"
    np.save(time_name, time)
    np.save(data_name, data)

Here I am saving and reloading the variables so I have them, but this isn't necessary.

In [27]:
temp = np.load("C:/Users/zdunkerton/Documents/GitHubData/Hurricane/Preprocessed/Temp_Surface_4xDaily.npy")
wind = np.load("C:/Users/zdunkerton/Documents/GitHubData/Hurricane/Preprocessed/UWind_Surface_4xDaily.npy")
humd = np.load("C:/Users/zdunkerton/Documents/GitHubData/Hurricane/Preprocessed/RelativeHumidity_Surface_4xDaily.npy")
press = np.load("C:/Users/zdunkerton/Documents/GitHubData/Hurricane/Preprocessed/Pressure_Surface_4xDaily.npy")

Read in the data on date at a time. For each data, read in the four values and stack them on top of each other. These layers all correspond to the same point in time. 

In [28]:
year = 0
data= []
while year < temp.shape[0]:
    day = 0
    while day < temp.shape[1]:
        temp1 = temp[year][day]
        temp1 = scaler.fit_transform(temp1)
        wind1 = wind[year][day]
        wind1 = scaler.fit_transform(wind1)
        humd1 = humd[year][day]
        humd1 = scaler.fit_transform(humd1)
        pres1 = press[year][day]
        pres1 = scaler.fit_transform(pres1)
        the_data = np.array([temp1,wind1,humd1,pres1])
        data.append(the_data)
        day+=1
    year+=1
data = np.array(data)
data.shape

(23360, 4, 73, 144)

In [29]:
np.save("Preprocessed_data.npy", data)

In [36]:
time = np.load("C:/Users/zdunkerton/Documents/GitHubData/Hurricane/Preprocessed/time.npy")

There are some days that have multiple hurricanes recorded. It is okay that times are only couted as having one hurricane, because the point is to see if there was a hurricane or not, not to determine size or location. 

In [101]:
df = pd.read_csv("./atlantic_hurricane.csv")
i = 0
date1 = datetime.datetime(1800,1,1,0,0)
hurricane_times = []
while i<len(df):
    the_time = df.iloc[i,3]
    if (the_time !=0) and (the_time !=600) and (the_time !=1200) and (the_time !=1800):
        if the_time <=900:
            df.iloc[i,3] = 600
        elif (the_time >900) and (the_time<=1500):
            df.iloc[i,3] = 1200
        elif the_time >1500:
            df.iloc[i,3] = 1800
    hurricane_time = datetime.datetime(int(str(df.iloc[i,2])[:4]), int(str(df.iloc[i,2])[4:6]),int(str(df.iloc[i,2])[6:8]),int(df.iloc[i,3]/100))
    converted_time = (hurricane_time-date1).total_seconds()/3600
    hurricane_times.append(converted_time)
    i+=1
labels = []
for t in time:
    if t in hurricane_times:
        labels.append(1)
        count+=1
    else:
        labels.append(0)

In [109]:
np.save("labels.npy", labels)